### Web Scraping 101 

#### Kindly ensure you have the legal rights to scrape and use data from a site before doing so. Propertypro is more flexible about this as seen in the terms and conditions page however Nigeria Property center is not. Check below for more: 

#### https://www.propertypro.ng/terms
#### https://nigeriapropertycentre.com/terms-of-use 

#### Import requests-html for making request to a website and scraping. re for regular expressions

In [1]:
import requests, re
from requests_html import HTMLSession

#### Make a request to the website and extract its content (page source)

In [2]:
r=requests.get("https://www.propertypro.ng/property-for-rent?search=ikorodu")
c=r.content

#### Create a Session and make a request to the website and extract its content (page source)

In [3]:
session = HTMLSession()
r = session.get(f'https://www.propertypro.ng/property-for-rent/in/lagos/ikorodu')

#### Find all properties on the page

In [7]:
properties = r.html.find('div.single-room-text')
properties

[<Element 'div' class=('single-room-text',)>,
 <Element 'div' class=('single-room-text',)>,
 <Element 'div' class=('single-room-text',)>,
 <Element 'div' class=('single-room-text',)>,
 <Element 'div' class=('single-room-text',)>,
 <Element 'div' class=('single-room-text',)>,
 <Element 'div' class=('single-room-text',)>,
 <Element 'div' class=('single-room-text',)>,
 <Element 'div' class=('single-room-text',)>,
 <Element 'div' class=('single-room-text',)>,
 <Element 'div' class=('single-room-text',)>,
 <Element 'div' class=('single-room-text',)>,
 <Element 'div' class=('single-room-text',)>,
 <Element 'div' class=('single-room-text',)>,
 <Element 'div' class=('single-room-text',)>,
 <Element 'div' class=('single-room-text',)>,
 <Element 'div' class=('single-room-text',)>,
 <Element 'div' class=('single-room-text',)>,
 <Element 'div' class=('single-room-text',)>,
 <Element 'div' class=('single-room-text',)>,
 <Element 'div' class=('single-room-text',)>]

#### To learn more about HTML tags check the link >>> https://www.w3schools.com/tags/default.asp

###### .

#### Collect property features on the page.For this add the index of the feature at the end of the code e.g. [0] for bed, [2] for bath.

#### This first method is not ideal because the location of the specific feature might change. For example Bed may not come first

In [8]:
properties[2].find('div.fur-areea')[0].text.split()

['4', 'beds', '4', 'baths', '4', 'Toilets']

#### This second method uses regular expressions and is a better way to collect feature information, because it checks for the feature before collecting. If the feature does not exist it will give a none. For more on regular expressions check https://www.w3schools.com/python/python_regex.asp 

In [19]:
re.findall("..bath",properties[2].find('div.fur-areea')[0].text)[0].strip()

'4 bath'

#### You can change div and class below to search for something else. 

In [20]:
properties[3].find('div.fur-areea')[0].text

'1 beds 1 baths 2 Toilets'

#### Websites typically have a structure which allows for easy automation. For example location and page number can eaasily be changed and the website will respond accordingly. Try changing the location below and page number to surulere and page 2 respectively.

In [14]:
https://www.propertypro.ng/property-for-rent?search=surulere&page=1

SyntaxError: invalid syntax (<ipython-input-14-7164a8e1540e>, line 1)

#### There is a slight challenge from above. You will need to get the total number of pages. This can be calculated using total number of items divided by number of listing on each page. The listing is written within a text so this has to be extracted using regular expressions. 

In [23]:
r.html.find('div.property-sale-number')[0].text

'Result 1 - 20 of 2345\nSort By\nMost Recent\nLowest Price\nHighest Price\nBeds'

In [24]:
re.findall("\d+",r.html.find('div.property-sale-number')[0].text)

['1', '20', '2345']

In [18]:
re.findall("\d+",r.html.find('div.property-sale-number')[0].text)[2]

'5344'

In [25]:
items = int(re.findall("\d+",r.html.find('div.property-sale-number')[0].text)[2])
listings = 20
page_nr = int(items/listings)
page_nr

117

#### The code below is used to extract the details on just one page per location. You can update the code to scrape multiple pages and locations or write your own code. Please as indicated in the instructions, do not scrape multiple locations or pages until off peak hours (>6pm) to avoid overloading the site.


In [ ]:
#session = HTMLSession()

In [26]:
l = []
LP1= "lekki/lekki-phase-1"
locations = ["gbagada", LP1, "ajah", "ikorodu", "surulere", "yaba", "ikeja"]

for place in locations:

    r = session.get(f'https://www.propertypro.ng/property-for-rent/in/lagos/'+place)

    items = int(re.findall("\d+",r.html.find('div.property-sale-number')[0].text)[2])
    listings = 20
    pages = int(items/listings) + 1

    for page in range(pages):

        r = session.get(f'https://www.propertypro.ng/property-for-rent/in/lagos/'+place+"?page="+str(page))
        properties = r.html.find('div.single-room-text')
        #base_url="https://www.propertypro.ng/property-for-rent?search="+place+ "&auto=&type=&bedroom=&max_price="
     
        for i in list(range(0,len(properties))):
            d={}
            d["location"] = place
            try:
                d["specific_location"] = properties[i].find('h4')[0].text
            except (IndexError,TypeError,AttributeError):
                d["location"] = None
            try:
                d["features"]= properties[i].find('div.fur-areea')[0].text 
            except (AttributeError,IndexError) as e:
                d["features"]= None
            try:
                d["bed"]= int(re.findall("..bed",properties[i].find('div.fur-areea')[0].text)[0].strip()[0][0])
            except (IndexError,TypeError,AttributeError,ValueError) as e:
                d["bed"]= 0 
            try:
                d["bath"]= int(re.findall("..bath",properties[i].find('div.fur-areea')[0].text)[0].strip()[0][0])
            except (IndexError,TypeError,AttributeError,ValueError) as e:
                d["bath"]= 0
            try:
                d["toilet"]= int(re.findall("..Toilet",properties[i].find('div.fur-areea')[0].text)[0].strip()[0][0])
            except (IndexError,TypeError,AttributeError,ValueError) as e:
                d["toilet"]= 0
            try:
                d["description"]= properties[i].find('h3.listings-property-title2')[0].text
            except (IndexError,TypeError,AttributeError) as e:
                d["description"]= None

            try:
                d["price"]= properties[i].find('h3.listings-price')[0].text.replace("₦ ","").replace(",","")
            except (IndexError,TypeError,AttributeError) as e:
                d["price"] = None
            l.append(d)

#### Convert output to dataframe

In [27]:
import pandas as pd
ld = pd.DataFrame(l)
ld

,location,specific_location,features,bed,bath,toilet,description,price
0,gbagada,"Diya Street, Ifako GbagadaIfako Gbagada Gbagad...",beds baths Toilets,0,0,0,Open Plan office space,1500000
1,gbagada,"Diya Street, Ifako GbagadaIfako Gbagada Gbagad...",beds baths 1 Toilets,0,0,1,Open Plan office space,1500000
2,gbagada,Phase 1Phase 1 Gbagada Lagos,4 beds 4 baths 4 Toilets,4,4,4,Tastefully Finished 4Bedroom Terraced Duplex i...,65000000
3,gbagada,Sawmill Gbagada Ifako Gbagada Gbagada Lagos,2 beds 2 baths 3 Toilets,2,2,3,5 units of Newly built 2 bedrooms up n ground ...,1200000
4,gbagada,Ifako Gbagada Gbagada Lagos,2 beds 2 baths 3 Toilets,2,2,3,Newly built executive 2bedroom duplex with mod...,1500000
...,...,...,...,...,...,...,...,...
46029,ikeja,Taiye Olowu Street Off Victoria Arobieke. Lekk...,1 beds 1 baths 2 Toilets,1,1,2,Luxury 1 (One) Bedroom Flat,None
46030,ikeja,Gbemisola House1 Obafemi Awolowo wayIkeja Ikej...,beds baths 10 Toilets,0,0,0,office space,10000000
46031,ikeja,off opebiIkeja Ikeja Lagos,3 beds 3 baths 3 Toilets,3,3,3,6 units of 3 bedroom flat,2000000
46032,ikeja,remi fani kayode streetIkeja Ikeja Lagos,4 beds 4 baths 5 Toilets,4,4,5,2 units of a terrace apartment+ bq,6000000


In [28]:
ld.to_csv("House_prices_lagos7.csv")

In [6]:
properties = r.html.find('ul.extra-listings')
properties

[<Element 'ul' class=('extra-listings',)>,
 <Element 'ul' class=('extra-listings',)>,
 <Element 'ul' class=('extra-listings',)>,
 <Element 'ul' class=('extra-listings',)>]

In [13]:
n = properties[3].text.split("|\n")

In [20]:
prop = [c.replace(" ", "") for c in n]

In [21]:
prop

['Agric',
 'AkuteAjuwon',
 'Apeka',
 'Ebute',
 'Ibeshe',
 'Igbogbo',
 'Ijede',
 'Ikorodu',
 'Ipakodo',
 'Isawo',
 'Jumofak',
 'Maya',
 'Odongunyan']